In [1]:
import pandas as pd
import urllib.request
import os

In [2]:
df_positive = pd.read_excel('数据提供：listing篡改_20220428.xlsx',engine='openpyxl',sheet_name='正确')
df_negative = pd.read_excel('数据提供：listing篡改_20220428.xlsx',engine='openpyxl',sheet_name='错误')

In [3]:
df_positive.head()

,ID,SiteInfoID,ListingUrl,Title,Price,类型,品类,code
0,2234,17,https://www.amazon.com/dp/B07B9VKWSL?_encoding...,Stud Finder Wall Scanner 4 in 1 Electric Wood...,25.99,正确,Pickle-Ball Equipment,D05567
1,2438,17,https://www.amazon.com/dp/B07GT37484?_encoding...,rockspace WiFi Extender 12 Gigabit Wireless S...,59.99,正确,Body Composition Monitors,D05575
2,2248,17,https://www.amazon.com/dp/B07GVDXW5D?_encoding...,Pickleball Paddles USAPA Pro Graphite Pickleba...,84.99,正确,Body Composition Monitors,D05575
3,2446,17,https://www.amazon.com/dp/B07M6Y7355?_encoding...,1080P Webcam with Microphone eMeet C960 Web Ca...,33.99,正确,Pet Supplies,D05588
4,2258,17,https://www.amazon.com/dp/B07R9XZSGL?_encoding...,3Axis Gimbal Stabilizer for iPhone 12 11 PRO M...,99.00,正确,Hobby RC Quadcopters & Multirotors,D05588


In [4]:
df_negative.head()

,ID,Asin,SiteInfoID,ListingUrl,Title,Price,类型,类目,code
0,2234,B07B9VKWSL,17,https://www.amazon.com/dp/B07B9VKWSL?_encoding...,Stud Finder Wall Scanner 4 in 1 Electric Wood...,25.99,错误,Stud Finders & Scanners,D05567
1,2438,B07GT37484,17,https://www.amazon.com/dp/B07GT37484?_encoding...,"rockspace WiFi Extender, Air Treatment Units o...",59.99,错误,Repeaters,D05575
2,2248,B07GVDXW5D,17,https://www.amazon.com/dp/B07GVDXW5D?_encoding...,Water Treatment Units or Filters，Pickleball Pa...,84.99,错误,Pickle-Ball Equipment,D05575
3,2446,B07M6Y7355,17,https://www.amazon.com/dp/B07M6Y7355?_encoding...,1080P Webcam with Microphone eMeet C960 Web Ca...,33.99,错误,Webcam Mounts,D05588
4,2258,B07R9XZSGL,17,https://www.amazon.com/dp/B07R9XZSGL?_encoding...,3Axis Gimbal Stabilizer for iPhone 12 11 PRO M...,99.00,错误,Professional Video Stabilizers,D05588


# listing文案防篡改

In [5]:
from sklearn.model_selection import train_test_split

x_train_p, x_test_p = train_test_split(df_positive,test_size=0.25,random_state=0)
x_train_n, x_test_n = train_test_split(df_negative,test_size=0.25,random_state=0)

train = pd.concat([x_train_p,x_train_n])
test = pd.concat([x_test_p,x_test_n])

train = train.rename(columns={"类型": "label","类目":"category"})
test = test.rename(columns={"类型": "label","类目":"category"})

#train['sentence1_key'] = train.apply(lambda row: row['category']+row['Title'])

train["category"] = train["category"].astype(str)

train['sentence1_key'] = train["category"] + ': ' + train["Title"]

test["category"] = test["category"].astype(str)

test['sentence1_key'] = test["category"] + ': ' + test["Title"]


In [6]:
print ("train size {}, test size{}".format(train.shape,test.shape))

train size (50, 11), test size(18, 11)


In [7]:
train[["label","sentence1_key"]].to_csv('./data/train.csv',index=False,encoding='utf-8')
test[["label","sentence1_key"]].to_csv('./data/test.csv',index=False,encoding='utf-8')

In [10]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()

prefix='haofang-datalab'

bucket = sess.default_bucket() 
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("./data/train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "test/test.csv")
).upload_file("./data/test.csv")

training_input_path = f's3://{sess.default_bucket()}/{prefix}/train/train.csv'
test_input_path = f's3://{sess.default_bucket()}/{prefix}/test/test.csv'

#git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.6.1'} # v4.6.1 is referring to the `transformers_version` you use in the estimator.

In [12]:
import sagemaker
from sagemaker.huggingface import HuggingFace

hyperparameters={'per_device_train_batch_size':4,
                 'per_device_eval_batch_size': 4,
                 'model_name_or_path': 'roberta-large',
                 'train_file':'/opt/ml/input/data/train/train.csv',
                 'validation_file':'/opt/ml/input/data/test/test.csv',
                 'test_file':'/opt/ml/input/data/test/test.csv',
                 'do_train': True,
                 'do_predict': True,
                 'do_eval': True,
                 'save_total_limit':3,
                 'num_train_epochs': 3,
                 'output_dir': '/opt/ml/model',
                 'num_train_epochs': 10,
                 'learning_rate': 5e-5,
                 'seed': 7,
                 'fp16': False,
                 'eval_steps': 1000,
                 }


# create the Estimator
huggingface_estimator = HuggingFace(
      entry_point='run_glue.py', # script
      source_dir='./', # relative path to example
      instance_type='ml.p3.2xlarge',
      instance_count=1,
      volume_size=500,
      transformers_version='4.6',
      pytorch_version='1.7',
      py_version='py36',
      role=role,
      base_job_name='haofang-roberta-large-epoch10',
      hyperparameters = hyperparameters
)

huggingface_estimator.fit({'train':training_input_path,'test':test_input_path})

2022-06-13 07:17:23 Starting - Starting the training job...
2022-06-13 07:17:49 Starting - Preparing the instances for trainingProfilerReport-1655104643: InProgress
.........
2022-06-13 07:19:15 Downloading - Downloading input data...
2022-06-13 07:19:46 Training - Downloading the training image.....................
2022-06-13 07:23:07 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-13 07:23:09,407 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-13 07:23:09,430 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-13 07:23:09,438 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-13 07:23:09,987 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_i

# deploy

In [13]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker

role = sagemaker.get_execution_role()

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   model_data=huggingface_estimator.model_data,  # path to your trained sagemaker model
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.6", # transformers version used
   pytorch_version="1.7", # pytorch version used
   py_version="py36", # python version of the DLC
)

In [14]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge"
)



------!

In [15]:
%%time
# example request, you always need to define "inputs"
data = {
   "inputs": "Desktop Photo Printers: Liene Photo Printer Paper amp Cartridge Cartridge Refill amp Change\xa0Modify Photo Paper 40 Pack"
}

# request
predictor.predict(data)

CPU times: user 13.2 ms, sys: 60 µs, total: 13.3 ms
Wall time: 695 ms


[{'label': '错误', 'score': 0.9999964833259583}]